In [190]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils import shuffle

In [191]:
class Loss(nn.Module):
    def __init__(self, lambda_u, lambda_L):
        super(Loss, self).__init__()
        self.lambda_u = lambda_u
        self.lambda_L = lambda_L

    def compute_f_loss(self, rating_mat, user_features, local_item_features, prob):
        non_zero_mask = (rating_mat != -1).type(torch.FloatTensor)
        ratings_predicted = torch.sigmoid(torch.mm(user_features, local_item_features.t()))
        
        diff = (ratings_predicted - rating_mat) ** 2
        prediction_error = torch.sum(diff * non_zero_mask)

        user_regularization = torch.sum(user_features ** 2)  ## regularization term for user features

        return (prediction_error + self.lambda_u * user_regularization) / (1 - prob)
    
    def compute_psi_loss(self, local_item_features, avg_item_features, prob):
        item_loss = torch.sum((local_item_features - avg_item_features) ** 2) ## loss term for item features

        return self.lambda_L * item_loss / prob

    def forward(self, rating_mat, user_features, local_item_features, avg_item_features):
        '''
        rating_mat: (num_users, num_items)
        user_features: (num_users_per_client, num_latent_factors)
        local_item_features: (num_items, num_latent_factors)
        avg_item_features: (num_items, num_latent_factors)
        '''
        non_zero_mask = (rating_mat != -1).type(torch.FloatTensor)
        ratings_predicted = torch.sigmoid(torch.mm(user_features, local_item_features.t()))
        
        diff = (ratings_predicted - rating_mat) ** 2
        prediction_error = torch.sum(diff * non_zero_mask)

        user_regularization = torch.sum(user_features ** 2)  ## regularization term for user features
        item_loss = torch.sum((local_item_features - avg_item_features) ** 2) ## loss term for item features

        loss = prediction_error + self.lambda_u * user_regularization + self.lambda_L * item_loss

        return loss, prediction_error

In [192]:
rating_df = pd.read_csv('ml-1m.inter', sep='\t')
rating_df.columns = ['user_id', 'item_id', 'rating', 'timestamp']
rating_df = shuffle(rating_df)
rating_df.head()

,user_id,item_id,rating,timestamp
916931,5539,107,3,960662169
46608,312,3617,3,976477183
504652,3104,1617,5,969556952
443640,2736,1244,2,973396870
410451,2462,1196,4,974168782


In [193]:
# Split the data into training and testing sets
ratio = 0.8
train_size = int(len(rating_df) * ratio)

aggregate_rating_matrix = rating_df.pivot_table(index='user_id', columns='item_id', values='rating', aggfunc='mean')  # transform the dataframe into a matrix
num_users, num_items = aggregate_rating_matrix.shape
rating_matrix = aggregate_rating_matrix.copy()
test_rating_matrix = aggregate_rating_matrix.copy()
for i in range(len(rating_df)):
    user_id = rating_df.iloc[i,0]
    item_id = rating_df.iloc[i,1]
    if i < train_size:
        test_rating_matrix.loc[user_id,item_id] = None
    else:
        rating_matrix.loc[user_id,item_id] = None

In [194]:
# normalize the ratings using min-max normalization
min_rating, max_rating = rating_df['rating'].min(), rating_df['rating'].max()
rating_matrix = rating_matrix.apply(lambda x: (x - min_rating) / (max_rating - min_rating))
rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)

In [195]:
num_users, num_items

(6040, 3706)

In [196]:
lr = 0.025
num_epochs = 100
latent_factors = 20
num_clients = 200
prob_threshold = 0.4
m = num_users // num_clients

In [197]:
# initializaiton

user_features = []
item_features = []
std = 0.01

for i in range(num_clients): # initialize user features and local item features
    user_features.append(torch.randn(m, latent_factors, requires_grad=True))  # multiplying std here will make the Tensor non-leaf, which will cause error
    item_features.append(torch.randn(num_items, latent_factors, requires_grad=True))
with torch.no_grad():
    for i in range(num_clients):
        user_features[i].data.mul_(std) # mul_ does not change requires_grad to False
        item_features[i].data.mul_(std)

avg_item_features = torch.randn(num_items, latent_factors).data.mul(std) # mul will change requires_grad to False
for i in range(num_clients):
    avg_item_features += item_features[i]
avg_item_features /= num_clients

# define the model
RFRec_loss = Loss(lambda_u=0.1, lambda_L=10)

client_optimizers = []
for i in range(num_clients):
    optimizer = optim.Adam([user_features[i], item_features[i]], lr=lr)
    client_optimizers.append(optimizer)

In [198]:
def train(epoch, rand_num, last_num):
    avg_loss = 0
    tmp = torch.zeros(item_features[0].shape)
    global avg_item_features

    # update
    if rand_num > prob_threshold:
        for i in range(num_clients):
            client_optimizers[i].zero_grad()
            if last_num > prob_threshold:
                loss = RFRec_loss.compute_f_loss(rating_matrix[i*m: (i+1)*m], user_features[i], item_features[i], prob_threshold)
            else:
                loss = RFRec_loss.compute_psi_loss(item_features[i], avg_item_features, prob_threshold)

            avg_loss += loss.item() / num_clients
            loss.backward(retain_graph=True)
            client_optimizers[i].step()
            tmp += item_features[i]
    else:
        avg_item_features = tmp / num_clients  # update the global item features

    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {:.4f}, '.format(epoch, avg_loss))

In [ ]:
last_num = 1
np.random.seed(42)
for epoch in range(num_epochs):
    rand_num = np.random.rand()
    train(epoch, rand_num, last_num)
    last_num = rand_num

Epoch: 0, Loss: 657.1425, 
Epoch: 10, Loss: 1019.7999, 
Epoch: 20, Loss: 655.3179, 
Epoch: 30, Loss: 658.4387, 
Epoch: 40, Loss: 653.4647, 
Epoch: 50, Loss: 0.0000, 
Epoch: 60, Loss: 144.1200, 
Epoch: 70, Loss: 0.0000, 
Epoch: 80, Loss: 664.4057, 
Epoch: 90, Loss: 650.6344, 


In [200]:
test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values)
print(test_rating_matrix.shape)

nonzero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)

torch.Size([6040, 3706])


In [201]:
def evaluate(matrix, user_features, item_features, mask):
    predicted_ratings = torch.sigmoid(torch.mm(user_features, item_features.t()))
    pred = (predicted_ratings * (max_rating - min_rating) + min_rating) * mask
    true_value = matrix * mask
    
    abs_error = torch.sum(torch.abs(pred - true_value))
    square_error = torch.sum((pred - true_value)**2)
    n_nonzero = torch.sum(mask)
    return abs_error, square_error, n_nonzero

In [202]:
MAE = MSE = num_nonzero = 0

for i in range(num_clients):
    abs_error, square_error, n_nonzero = evaluate(test_rating_matrix[i*m: (i+1)*m], user_features[i], item_features[i], nonzero_mask[i*m: (i+1)*m])
    MAE += abs_error
    MSE += square_error
    num_nonzero += n_nonzero

MAE /= num_nonzero
RMSE = torch.sqrt(MSE / num_nonzero)
print("MAE: ", MAE.data.numpy())
print("RMSE: ", RMSE.data.numpy())

MAE:  1.0195738
RMSE:  1.2548442
